In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import disk, opening, closing
import glob
import time
import h5py

In [8]:
# Read input image and resize them
folder_in = 'imgs_samples/'
folder_out = 'imgs_samples_resized/'
images = sorted(glob.glob(folder_in + '*.jpg'))
resized_imgs_path = sorted(glob.glob(folder_out + '*.jpg'))
n = 0
for img_path in images:
    cv2.imwrite(folder_out + 'resized' + str(n) + '.jpg', cv2.resize(cv2.imread(img_path), (0,0), fx=0.5, fy=0.5));
    n += 1

In [1]:
ILASTIK_PATH = '/Users/fabianprofeta/Desktop/ilastik-1.4.0.post1-OSX.app/Contents/ilastik-release/run_ilastik.sh'
MODEL='bees.ilp'
IMG = ''
OUTPUT_FOLDER = './imgs_samples_mask/{nickname}_mask.h5'
for img_path in resized_imgs_path:
    IMG += img_path + ' '
# we call ilastik as headless operation on the pc
!$ILASTIK_PATH  --headless \
                --project=$MODEL \
                --output_filename_format=$OUTPUT_FOLDER \
                $IMG

NameError: name 'resized_imgs_path' is not defined

In [5]:
# load the .h5 file and save them as png
filenames = sorted(glob.glob('imgs_samples_mask/*.h5'))
n = 0
masks = []
times = []
for filename in filenames:
    start_time = time.time()
    with h5py.File(filename, "r") as f:
        a_group_key = list(f.keys())[0]
        data = list(f[a_group_key])
        ds_arr = f[a_group_key][:,:,0]  # returns as a numpy array
        cv2.imwrite('imgs_samples_mask/resized' + str(n) + '_mask.png', ds_arr)
        masks.append(cv2.imread('imgs_samples_mask/resized' + str(n) + '_mask.png', cv2.IMREAD_GRAYSCALE))
        n += 1
print('Average time taken: ', round(np.mean(times), 2), 's')

Average time taken:  2.98 s


In [6]:
# morphological operations
final_masks = []
times = []
for mask in masks:
    start_time = time.time()
    mask_open = opening(mask, disk(9))
    mask_closed = closing(mask_open, disk(9))
    final_masks.append(mask_closed)
    times.append(round(time.time() - start_time, 2))
print('Average time taken: ', round(np.mean(times), 2), 's')


Average time taken:  4.05 s


In [22]:
def elongation(m):
    x = m['mu20'] + m['mu02']
    y = 4 * m['mu11']**2 + (m['mu20'] - m['mu02'])**2
    return (x + y**0.5) / (x - y**0.5)

# now we can find the contours
imgs_contour = []
n = 0
resized_imgs = []
times = []
for img_path in resized_imgs_path:
    resized_imgs.append(cv2.imread(img_path))

for mask, input_image in zip(final_masks, resized_imgs):
    start_time = time.time()
    img_contour = (mask).astype(np.uint8) #convert to uint8 because findContours only accepts uint8
    contours, _ = cv2.findContours(img_contour, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #find contours
    contour = max(contours, key=cv2.contourArea) #get the biggest contour
    img_contour = cv2.cvtColor(img_contour, cv2.COLOR_GRAY2BGR) #convert to BGR because drawContours only accepts BGR
    cv2.drawContours(input_image, contour, -1, (255, 0, 0), 15) #draw the biggest contour

    # find the centroid of the contour
    M = cv2.moments(contour)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    # print('Elongation of img' + str(n) + ': ', elongation(M))
    # plot it on the image
    cv2.circle(input_image, (cX, cY), 10, (255, 0, 0), -1)
    cv2.putText(input_image, "centroid", (cX - 30, cY - 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    ellipse = cv2.fitEllipse(contour)
    cv2.ellipse(input_image, ellipse, color=(0,255,0), thickness=15)
    print(ellipse)

    # rect = cv2.minAreaRect(contour)
    # box = cv2.boxPoints(rect)
    # box = np.intp(box)
    # cv2.drawContours(input_image, [box], 0, (0, 0, 255), 2)

    cv2.imwrite('imgs_samples_contour/contour' + str(n) + '.jpg', cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB))
    times.append(round(time.time() - start_time, 2))
    n += 1

print('Average time taken: ', round(np.mean(times), 2), 's')

((865.2841796875, 424.0394287109375), (842.6317138671875, 942.6292724609375), 88.46150207519531)
((762.183837890625, 158.50216674804688), (884.5503540039062, 1316.9501953125), 25.577730178833008)
((868.1810302734375, 410.31317138671875), (867.7225341796875, 906.8765258789062), 66.42987823486328)
((775.7647705078125, 261.59405517578125), (855.0314331054688, 1160.782958984375), 31.697193145751953)
((769.8652954101562, 220.76947021484375), (865.4181518554688, 1286.22119140625), 24.212339401245117)
((744.3483276367188, 184.57211303710938), (855.3819580078125, 1269.8563232421875), 21.515071868896484)
((760.0075073242188, 171.47825622558594), (851.43017578125, 1354.1614990234375), 25.833498001098633)
((780.7593383789062, 301.1068115234375), (851.7034301757812, 1004.2666015625), 49.21540832519531)
((821.33154296875, 354.46990966796875), (745.7223510742188, 1070.68115234375), 72.12784576416016)
((732.0276489257812, 333.35626220703125), (839.2943115234375, 982.4647827148438), 55.346412658691406